In [ ]:
# Install Package

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
# Import Library

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import os
import pandas as pd
import progressbar

In [ ]:
# Set GPU environment
device = torch.device("cuda:0")

In [ ]:
# Install KoBert
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
Path_1 = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/nonTour_df'
Mall_super_list = os.listdir(Path_1)
Mall_super_list.remove('Complete_list.pkl')
Mall_super_list.remove('Check_list.pkl')

total_df = pd.DataFrame()

### Data Load

In [ ]:
# 투어몰 가지고 데이터셋 만들기.

for i in Mall_super_list[60:]:
  Video_Path = Path_1 + f'/{i}'
  video_list = os.listdir(Video_Path)
  for video in video_list:
    try:
      video_path = Video_Path + f'/{video}'
      temp_df = pd.read_csv(video_path)
      total_df = pd.concat([total_df, temp_df], ignore_index=True)
      total_df.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_Test.tsv', index = False, sep = '\t')

    except:
      print(f'ERROR!! Not removed. {video_path}')
total_df.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_Test.tsv', index = False, sep = '\t')

In [ ]:
total_df2 = total_df[['chat', 'GPT_Sentiment_Analysis']]
total_df2.dropna(axis=0, inplace=True)

<ipython-input-51-e0572aa8d037>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df2.dropna(axis=0, inplace=True)


In [ ]:
total_df2.reset_index(inplace=True, drop=True)
total_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142599 entries, 0 to 142598
Data columns (total 2 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   chat                    142599 non-null  object
 1   GPT_Sentiment_Analysis  142599 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [ ]:
total_df2.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_train_2.tsv', index = False, sep = '\t')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_test_3.tsv', sep = '\t')
test = test[['chat', 'GPT_Sentiment_Score']]

In [ ]:
test.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_test_4.tsv', index = False, sep = '\t')

In [ ]:
test = test[~test['chat'].str.contains('\t')]

In [ ]:
test.reset_index(inplace=True, drop=True)
test

In [ ]:
test.reset_index(inplace=True, drop=True)
test

In [ ]:
test[test['GPT_Sentiment_Score'] == 'Positive']

,chat,GPT_Sentiment_Analysis,GPT_Sentiment_Score


In [ ]:
mapping_dict = dict({'Very Positive' : 2, 'Positive' : 1, 'Neutral' : 0, 'Negative' : -1, 'Very Negative' : -2})

In [ ]:
mapping_dict = dict({'Very Positive' : 4, 'Positive' : 3, 'Neutral' : 2, 'Negative' : 1, 'Very Negative' : 0})

In [ ]:
test['GPT_Sentiment_Score'] = test['GPT_Sentiment_Analysis'].map(lambda x: mapping_dict.get(x))

In [ ]:
test.dropna(inplace=True)
test.reset_index(inplace=True, drop=True)
test

In [ ]:
test

In [ ]:
test.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_train_3.tsv', index = False, sep = '\t')

# 데이터 시작

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_train_4.tsv', sep = '\t')


In [ ]:
mapping_dict = dict({-2 : 0, -1 : 1, 0 : 2, 1 : 3, 2 : 4})

In [ ]:
test['GPT_Sentiment_Score_2'] = test['GPT_Sentiment_Score'].map(lambda x: mapping_dict.get(x))

In [ ]:
test = test[['chat', 'GPT_Sentiment_Score_2']]
test
test.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_train_5.tsv', index = False, sep = '\t')

In [ ]:
test.GPT_Sentiment_Score.unique()

array([ 0,  1,  2, -1, -2])

In [ ]:
# Ready Data

dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_train_5.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_test_5.tsv', field_indices=[0,1], num_discard_samples=1)

In [ ]:
for i in range(len(dataset_train)):
  if 'Positive' in dataset_train[i][1]:
    print(dataset_train[i])

In [ ]:
# Set BERT Tokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:

#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


NameError: ignored

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
# KoBERT Modeling.

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, ##Categories. 0 = Very Negative, 1 = Negative, 2 = Neutral, 3 = Positive, 4 = Very Positive.
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

NameError: ignored

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

### 모델 학습

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
PATH = '/content/drive/MyDrive/Project/LiveCommerse/OpenAI/KoBERT/' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'KoBERT_GPT-turn.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

NameError: ignored

### 모델 테스트.

In [ ]:
PATH = '/content/drive/MyDrive/Project/LiveCommerse/OpenAI/KoBERT/'
model = torch.load(PATH + 'KoBERT_GPT-turn.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt')) 

<All keys matched successfully>

In [ ]:
Path_1 = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/nonTour_df'
Path_2 = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Tour_only_df'
Mall_super_list = os.listdir(Path_1)
Mall_tour_list = os.listdir(Path_2)
Mall_super_list.remove('Complete_list.pkl')
Mall_super_list.remove('Check_list.pkl')
Mall_tour_list.remove('Complete_list.pkl')
Mall_tour_list.remove('Check_list.pkl')

In [ ]:
for i in Mall_super_list[-13:]:
  print(i)

시바산 스토어_65412
EXITour_74500
단비마음_97717
BC24_61042
제주김만복_73162
사가브_74425
비욘 VYON_72138
소셜 굿즈_71428
B2Model_76556
김효석몰_80256
호놀룰루컴퍼니_86290
관광기업지원센터_83618
SUSIE_85550


In [ ]:
Mall_super_list

In [ ]:
test.to_csv('/content/drive/MyDrive/Project/LiveCommerse/OpenAI/DataFrame_train_3.tsv', index = False, sep = '\t')

In [ ]:
import pickle

In [ ]:
mapping_dict_2 = dict({'Very Positive' : 4, 'Positive' : 3, 'Neutral' : 2, 'Negative' : 1, 'Very Negative' : 0})
mapping_dict_1 = dict({'Very Positive' : 2, 'Positive' : 1, 'Neutral' : 0, 'Negative' : -1, 'Very Negative' : -2})
error_list = []
for i in Mall_tour_list:
  Video_Path = Path_2 + f'/{i}'
  video_list = os.listdir(Video_Path)

  for j in video_list:
    if 'csv' not in j:
      video_list.remove(j)
      print(f'remove : {j}')

  for k in video_list:
    if len(k) > 17:
      video_list.remove(k)
      print(f'remove : {k}')

  for video in video_list:
    print(video)
    try:
      video_path = Video_Path + f'/{video}'
      print(video_path)
      df = pd.read_csv(video_path)
      try:
        temp_df = df[['name', 'chat', 'GPT_Sentiment_Analysis']]
      except:
        continue
      temp_df = temp_df[~temp_df['chat'].str.contains('\t')]
      temp_df['GPT_Sentiment_Score_1'] = temp_df['GPT_Sentiment_Analysis'].map(lambda x: mapping_dict_1.get(x))
      temp_df['GPT_Sentiment_Score_2'] = temp_df['GPT_Sentiment_Analysis'].map(lambda x: mapping_dict_2.get(x))
      temp_df.dropna(inplace=True)
      temp_df.reset_index(inplace=True, drop=True)
      temp_df['GPT_Sentiment_Score_1'] = temp_df['GPT_Sentiment_Score_1'].astype(int)
      temp_df['GPT_Sentiment_Score_2'] = temp_df['GPT_Sentiment_Score_2'].astype(int)
      temp_df.to_csv(video_path, index=False)
      temp_df.to_csv('./temp.tsv', index = False, sep = '\t')
      new_test = nlp.data.TSVDataset('./temp.tsv', field_indices=[1,4], num_discard_samples=1)
      test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
      test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()
        temp_df.loc[batch_id, 'KoBERT_Sentiment_Score_1'] = prediction
      temp_df.to_csv(video_path, index=False)
    except:
      error_list.append(Video_Path + f'/{video}')
      print('Error Occur! \n')
      print(Video_Path + f'/{video}')

with open('/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/tour_error_2.pkl', 'wb') as f:
  pickle.dump(error_list, f)

In [ ]:
new_test = nlp.data.TSVDataset('/content/drive/MyDrive/test.tsv', field_indices=[1,2], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

FileNotFoundError: ignored

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  print(batch_id + "번째 문장의 분류 예측값은" + prediction + "입니다.")